In [ ]:
import random

class Territory:
    def __init__(self, name):
        self.name = name
        self.owner = None
        self.troops = 0
        self.adjacent_territories = []

    def add_adjacent(self, other):
        self.adjacent_territories.append(other)

class Player:
    def __init__(self, name):
        self.name = name
        self.territories = []

    def add_territory(self, territory):
        territory.owner = self
        self.territories.append(territory)

    def remove_territory(self, territory):
        territory.owner = None
        self.territories.remove(territory)

In [ ]:
class Game:
    def __init__(self, players, territories):
        self.players = players
        self.territories = territories
        self.current_player = players[0]

    def next_turn(self):
        idx = self.players.index(self.current_player)
        self.current_player = self.players[(idx + 1) % len(self.players)]

    def attack(self, attacker, defender):
        if attacker.owner != self.current_player:
            print(f"{self.current_player.name} cannot attack from a territory they don't own.")
            return
        if defender not in attacker.adjacent_territories:
            print(f"{attacker.name} is not adjacent to {defender.name}")
            return
        if attacker.troops <= 1:
            print("Not enough troops to attack.")
            return

        attacker_roll = sorted([random.randint(1, 6) for _ in range(attacker.troops - 1)], reverse=True)
        defender_roll = sorted([random.randint(1, 6) for _ in range(defender.troops)], reverse=True)
        print(f"{attacker.name} rolls: {attacker_roll}")
        print(f"{defender.name} rolls: {defender_roll}")

        for a, d in zip(attacker_roll, defender_roll):
            if a > d:
                defender.troops -= 1
            else:
                attacker.troops -= 1

        if defender.troops <= 0:
            print(f"{self.current_player.name} conquered {defender.name}!")
            defender.owner.remove_territory(defender)
            self.current_player.add_territory(defender)
            defender.troops = attacker.troops // 2
            attacker.troops -= defender.troops

    def check_winner(self):
        for player in self.players:
            if len(player.territories) == len(self.territories):
                print(f"{player.name} has conquered the map and wins!")
                return player
        return None

In [ ]:
territories = [Territory("Territory " + str(i)) for i in range(1, 6)]
players = [Player("Player 1"), Player("Player 2")]

# Manually set up adjacent territories
territories[0].add_adjacent(territories[1])
territories[1].add_adjacent(territories[0])
territories[1].add_adjacent(territories[2])
territories[2].add_adjacent(territories[1])
territories[2].add_adjacent(territories[3])
territories[3].add_adjacent(territories[2])
territories[3].add_adjacent(territories[4])
territories[4].add_adjacent(territories[3])

# Distribute territories and troops
for i, territory in enumerate(territories):
    players[i % 2].add_territory(territory)
    territory.troops = random.randint(3, 10)

# Game loop
game = Game(players, territories)

while not game.check_winner():
    print(f"{game.current_player.name}'s turn.")
    # Simulate an attack (choose territories to attack)
    attacker = game.current_player.territories[0]
    defender = attacker.adjacent_territories[0]
    game.attack(attacker, defender)
    game.next_turn()